In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollator
from datasets import load_dataset
import ast
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

e:\Conda\envs\news_summ\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка предварительно обученной модели BERT
MODEL = 'bert-base-multilingual-uncased'
model = BertForSequenceClassification.from_pretrained(MODEL, num_labels=2, return_dict=True)
tokenizer = BertTokenizer.from_pretrained(MODEL)

DATA = './data.csv'
BATCH_SIZE = 3

dataset = load_dataset('csv', data_files=DATA)

train_val = dataset["train"].train_test_split(
    test_size=0.1, shuffle=False, seed=42
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [33]:
def tokenize_and_align_labels(examples):

    #inputs = ast.literal_eval(examples['Normalized_news'][0])
    #print(examples['Normalized_news'])
    print('a')
    inputs = [ast.literal_eval(ex) for ex in examples['Normalized_news']]
    print(inputs)
    encoded_input = tokenizer(
        inputs,
        max_length=512,
        padding='max_length',
        truncation=True,
        add_special_tokens=True,
        is_split_into_words=True,
        #return_tensors='pt',
    )
    

    #pred = ast.literal_eval(examples['Class'])

    encoded_input["labels"] = [ast.literal_eval(ex) for ex in examples['Class']]#[ast.literal_eval(ex) for ex in examples['Class']]
    print(encoded_input["labels"])
    print(f"Return: {encoded_input}")
    return encoded_input

In [34]:
train_data = (
    train_val["train"].shuffle().map(tokenize_and_align_labels, batched=True, remove_columns=['Unnamed: 0', 'Original_news', 'Normalized_news', 'Class', 'Original_summary'])#, batched=True)
)
#train_data = train_data.rename_column('Class', 'labels')
#train_data = [tokenize_and_align_labels(input) for input in train_val['train']]
#val_data = [tokenize_and_align_labels(input) for input in train_val['test']]
val_data = (
    train_val["test"].shuffle().map(tokenize_and_align_labels, batched=True, remove_columns=['Unnamed: 0', 'Original_news', 'Normalized_news', 'Class', 'Original_summary'])#, batched=True)
)

a
[['вступление должность возникнуть конфликт пресса оборонный отрасль сенатор республиканец разведслужба', 'сми сообщать предвыборный штаб вступить сговор российский спецслужба быть компромат', 'сам избрать президент утверждать глава нацразведка звонить сказать сведение компромат ложный'], ['бороться облысение викинг использовать примочка гусиный помета', 'древнегреческий медик лучший лекарство облысение считать голубиный помет смешивать хрен тмин крапива', 'египетский рецепт 5000-летний давность нужный быть сжечь колючок еж смесить масло добавить масса мед алебастр красный охра обрезок ногть', 'получиться снадобей следовать смазывать облысеть область', 'вопрос облысение волновать мужчина самый пора обиход войти зеркало', 'пример быть одержий мысль вернуть шевелюра быть готовый', 'лавровый венок носить отдать дань римский традиция маскировка проплешин', 'момент знакомство быть лысый', 'возлюбленная сила пытаться помочь сохранить шевелюра предложить домашний средство основа измельчить 


Map: 100%|██████████| 13/13 [00:00<00:00, 28.53 examples/s]

[[1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 



Map: 100%|██████████| 2/2 [00:00<00:00, 33.83 examples/s]

a
[['самый маленький рост время финансовый кризис 2009 год', 'последний месяц прошлый год китайский вырасти', '2014 год экономика второй величина мир вырасти год достичь заявить цель'], ['грузин окружение лишить вид жительство прорыв украинско польский граница сентябрь', 'день охранять грузинский соотечественник данные генпрокуратура миграционный служба заниматься выдворение гражданин получить разрешение пребывание время нахождение глава одесский областной администрация ценный деятель культура', 'генпрокурор заявить человек готовить государственный переворот', 'ведомство заявить миграционный служба действовать соответствие законодательство решение принудительный возвращение мочь быть принять иностранец лицо гражданство нарушать законодательство правовой статус противоречить интерес обеспечение национальный безопасность страна', 'гражданин число подполковник грузинский армия отставка отправить октябрь', 'интервью рассказывать делать происходить выдворение', 'генеральный прокуратура врем

In [35]:
args = TrainingArguments(
    MODEL,
    # evaluation_strategy = "epoch",
    logging_strategy = 'epoch',
    save_total_limit=1,
    save_strategy='epoch',
    warmup_steps=250,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    lr_scheduler_type='linear',
    metric_for_best_model='f1_m',
    seed=42
)

In [25]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)

    # Remove ignored index (special tokens) and flatten the output
    true_predictions = [p for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label) if l != -100 ]
    true_labels = [l for prediction, label in zip(predictions, labels) for (p, l) in zip(prediction, label) if l != -100]
    precision, recall, f1, _ = precision_recall_fscore_support(y_true=true_labels, y_pred=true_predictions, average=None)
    f1_macro = f1_score(y_true=true_labels, y_pred=true_predictions, average="macro")
    return {
        'precision': list(precision),
        'recall': list(recall),
        'f1': list(f1),
        'f1 macro': f1_macro
    }

In [36]:
data_collator = DataCollator(tokenizer)

In [37]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

training_output = trainer.train()

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).